In [35]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
import os 
from PIL import Image

In [36]:
def get_jpg_path(file_dir='datasets\\birds\\train\\'):
    path=[]
    clazz=[]
    for root, dirs, files in os.walk(file_dir): 
        if len(dirs)==0:
            cls=root[-3:]
            cls=[int(cls)]*len(files)
            ph=[root+"\\"+p for p in files]
            path.extend(ph)
            clazz.extend(cls)
    return path,clazz
path,clazz=get_jpg_path('datasets\\birds\\train\\')

In [37]:
def onehot_label(clazz):
    index=0
    is_insight=set()
    hot=[0]*90
    onehot=[]
    for i in clazz:
        if i not in is_insight:
            is_insight.add(i)
            hot=[0]*90
            hot[index]=1
            index+=1
        onehot.append(hot)
    onehot=np.asarray(onehot)
    return onehot

In [48]:
onehot=onehot_label(clazz)
onehot[0:35]

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [39]:
def reszie_jpg(path):
    imgs=np.zeros(shape=(3830,224,224,3))
    index=0
    for p in path:
        im=Image.open(p)
        im=im.resize((224,224))
        npim=np.asarray(im)
        if npim.shape!=(224,224,3):
            print(npim.shape,index)
        else:
            imgs[index]=npim
        index+=1
    return imgs

In [40]:
imgs=reszie_jpg(path)
imgs.shape

(224, 224) 84
(224, 224) 375
(224, 224) 1358
(224, 224) 1800
(224, 224) 1942
(224, 224) 2218


(3830, 224, 224, 3)

In [52]:
model = Sequential()
# BLOCK 1
model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block1_conv1', input_shape = (224, 224, 3)))   
model.add(MaxPooling2D(pool_size = (4, 4), strides = (4, 4), name = 'block2_pool1'))
model.add(Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block1_conv2'))
model.add(MaxPooling2D(pool_size = (4, 4), strides = (4, 4), name = 'block1_pool2'))
 
# BLOCK2
model.add(Conv2D(filters = 256, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block2_conv1'))
model.add(MaxPooling2D(pool_size = (4, 4), strides = (4, 4), name = 'block2_pool3'))
model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block2_conv2'))
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2,2), name = 'block2_pool4'))
 
# BLOCK3
# model.add(Conv2D(filters = 256, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block3_conv1'))   
# # model.add(Conv2D(filters = 256, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block3_conv2'))
# model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block3_conv3'))
# model.add(MaxPooling2D(pool_size = (4, 4), strides = (4, 4), name = 'block3_pool'))
 
# # BLOCK4
# model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block4_conv1'))   
# model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block4_conv2'))
# model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block4_conv3'))
# model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), name = 'block4_pool'))
 
# # BLOCK5
# model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block5_conv1'))   
# model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block5_conv2'))
# model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block5_conv3'))
# model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), name = 'block5_pool'))
 
model.add(Flatten())
model.add(Dense(1024, activation = 'relu', name = 'fc1'))
model.add(Dropout(rate=0.5))
model.add(Dense(1024, activation = 'relu', name = 'fc2'))
model.add(Dropout(rate=0.5))
model.add(Dense(90, activation = 'softmax', name = 'prediction'))

In [53]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block2_pool1 (MaxPooling2D)  (None, 56, 56, 64)        0         
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 56, 56, 128)       73856     
_________________________________________________________________
block1_pool2 (MaxPooling2D)  (None, 14, 14, 128)       0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 14, 14, 256)       295168    
_________________________________________________________________
block2_pool3 (MaxPooling2D)  (None, 3, 3, 256)         0         
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 3, 3, 512)         1180160   
__________

In [57]:
model.compile(optimizer=Adam(0.001),
              #loss='mse',
              loss='categorical_crossentropy',
              #loss='binary_crossentropy',
              metrics=['accuracy'])

In [58]:
from sklearn.model_selection import train_test_split
xtr,xte,ytr,yte=train_test_split(imgs,onehot,test_size=0.3,random_state=1)

In [59]:
model.fit(xtr, ytr, epochs=100)

Epoch 1/100
2681/2681 [==============================] - 7s 3ms/sample - loss: 15.9017 - acc: 0.0134
Epoch 2/100
2681/2681 [==============================] - 7s 3ms/sample - loss: 15.9017 - acc: 0.0134
Epoch 3/100
2681/2681 [==============================] - 7s 3ms/sample - loss: 15.9077 - acc: 0.0131
Epoch 4/100
2681/2681 [==============================] - 7s 3ms/sample - loss: 15.9017 - acc: 0.0134
Epoch 5/100
2681/2681 [==============================] - 7s 3ms/sample - loss: 15.8957 - acc: 0.0138
Epoch 6/100
2681/2681 [==============================] - 7s 3ms/sample - loss: 15.9137 - acc: 0.0127
Epoch 7/100
2681/2681 [==============================] - 7s 3ms/sample - loss: 15.9077 - acc: 0.0131
Epoch 8/100
2681/2681 [==============================] - 7s 3ms/sample - loss: 15.9257 - acc: 0.0119
Epoch 9/100
2681/2681 [==============================] - 7s 3ms/sample - loss: 15.9077 - acc: 0.0131
Epoch 10/100
2681/2681 [==============================] - 7s 3ms/sample - loss: 15.9077 - a

2681/2681 [==============================] - 7s 3ms/sample - loss: 15.9137 - acc: 0.0127
Epoch 82/100
2681/2681 [==============================] - 7s 3ms/sample - loss: 15.9017 - acc: 0.0134
Epoch 83/100
2681/2681 [==============================] - 7s 3ms/sample - loss: 15.8957 - acc: 0.0138
Epoch 84/100
2681/2681 [==============================] - 7s 3ms/sample - loss: 15.8836 - acc: 0.0145
Epoch 85/100
2681/2681 [==============================] - 7s 3ms/sample - loss: 15.9077 - acc: 0.0131
Epoch 86/100
2681/2681 [==============================] - 7s 3ms/sample - loss: 15.9197 - acc: 0.0123
Epoch 87/100
2681/2681 [==============================] - 7s 3ms/sample - loss: 15.9017 - acc: 0.0134
Epoch 88/100
2681/2681 [==============================] - 7s 3ms/sample - loss: 15.9017 - acc: 0.0134
Epoch 89/100
2681/2681 [==============================] - 7s 3ms/sample - loss: 15.9017 - acc: 0.0134
Epoch 90/100
2681/2681 [==============================] - 7s 3ms/sample - loss: 15.9077 - acc: 

In [46]:
rs=model.predict(imgs[0:10])
for i in range(len(rs)):
    print(rs[i],onehot[i])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.